In [1]:
import pandas as pd

with open('../data/SaleDataset.csv', 'r') as f:
    data = pd.read_csv(f)

In [2]:
import pandas as pd
import numpy as np

def Convert_Ordinal(data:pd.DataFrame) -> pd.DataFrame:

    out = data.copy()

    #MSSubClass should be categorical
    mapping = {
        20:'1-STORY 1946 & NEWER ALL STYLES',
        30:'1-STORY 1945 & OLDER',
        40:'1-STORY W/FINISHED ATTIC ALL AGES',
        45:'1-1/2 STORY - UNFINISHED ALL AGES',
        50:'1-1/2 STORY FINISHED ALL AGES',
        60:'2-STORY 1946 & NEWER',
        70:'2-STORY 1945 & OLDER',
        75:'2-1/2 STORY ALL AGES',
        80:'SPLIT OR MULTI-LEVEL',
        85:'SPLIT FOYER',
        90:'DUPLEX - ALL STYLES AND AGES',
        120:'1-STORY PUD (Planned Unit Development) - 1946 & NEWER',
        150:'1-1/2 STORY PUD - ALL AGES',
        160:'2-STORY PUD - 1946 & NEWER',
        180:'PUD - MULTILEVEL - INCL SPLIT LEV/FOYER',
        190:'2 FAMILY CONVERSION - ALL STYLES AND AGES'
    }
    out['MSSubClass'] = out['MSSubClass'].replace(mapping)

    #NA is not value, but legit NA; Doesn't matter with later .fillna(0)
    #Quality and Condition
    mapping = {
        'Ex':5,
        'Gd':4,
        'TA':3,
        'Fa':2,
        'Po':1,
        'NA':-1,
    }
    col_apply = [
        'ExterQual', 'ExterCond',
        'BsmtQual', 'BsmtCond',
        'HeatingQC',
        'KitchenQual',
        'GarageQual', 'GarageCond',
        'FireplaceQu'
    ]
    for col in col_apply: 
        out[col] = out[col].replace(mapping)
        out[col] = out[col].fillna(-1)
        out[col] = out[col].astype(int)

    #Basement Exposure
    mapping = {
        'Gd':3,
        'Av':2,
        'Mn':1,
        'No':0
    }
    out['BsmtExposure'] = out['BsmtExposure'].replace(mapping)
    out['BsmtExposure'] = out['BsmtExposure'].fillna(-1)
    out['BsmtExposure'] = out['BsmtExposure'].astype(int)

    #Basement Finish - Retired
    '''mapping = {
        'GLQ':5,
        'ALQ':4,
        'BLQ':3,
        'Rec':2,
        'LwQ':1,
        'Unf':0,
    }
    col_apply = ['BsmtFinType1', 'BsmtFinType2']
    for col in col_apply: 
        out[col] = out[col].replace(mapping)
        out[col] = out[col].fillna(-1)
        out[col] = out[col].astype(int)'''

    #Deductions
    mapping = {
        'Typ':-1,
        'Min1':1,
        'Min2':2,
        'Mod':3,
        'Maj1':4,
        'Maj2':5,
        'Sev':6,
        'Sal':7
    }
    out['Functional'] = out['Functional'].replace(mapping).astype('int')

    #Garage Finsish
    mapping = {
        'Fin':1,
        'RFn':0.5,
        'Unf':0
    }
    out['GarageFinish'] = out['GarageFinish'].replace(mapping).astype('float')
    out['GarageFinish'] = out['GarageFinish'].fillna(0)

    #PavedDrive; Penalized for non as most have one
    mapping = {
        'Y':0,
        'P':0.5,
        'N':1
    }
    out['NoPavedDrive'] = out['PavedDrive'].replace(mapping).astype(float)
    out = out.drop(columns=['PavedDrive'])

    #Fence Quality
    mapping = {
        'GdPrv':4,
        'MnPrv':3,
        'GdWo':2,
        'MnWw':1,
        'NA':-1
    } 
    out['Fence'] = out['Fence'].replace(mapping)
    out['Fence'] = out['Fence'].fillna(-1)
    out[col] = out[col].astype(int)

    return out

def Noncategorical_Dummies(data:pd.DataFrame) -> pd.DataFrame:

    out = data.copy()

    out['CentralAir'] = np.where(out['CentralAir'] == 'N', 0, 1)

    out['Remodeled'] = np.where(out['YearRemodAdd'] == out['YearBuilt'], 0, 1)
    
    out['MasonryVeneer'] = np.where(out['MasVnrType'].isna(), 0, 1)
    out = out.drop(columns=['MasVnrArea', 'MasVnrType'])

    out['AlleyAccess'] = np.where(out['Alley'].isna(), 0, 1)
    out = out.drop(columns=['Alley'])

    # 'Has_' Prefix indicates var, that are both dummies but also have rankings/values attached 
    out['Has_Basement'] = np.where(out['BsmtExposure'].isna(), 0, 1)

    out['Has_Garage'] = np.where(out['GarageType'].isna(), 0, 1)

    out['Has_Deduction'] = np.where(out['Functional'] == 'Typ', 0, 1)

    out['Has_StreetAccess'] = np.where(out['LotFrontage'].isna(), 0, 1)

    out['Has_Fireplace'] = np.where(out['Fireplaces'] == 0, 0, 1)

    out['Has_Porch'] = np.where(out['OpenPorchSF'] == 0, 0, 1)

    out['Has_Deck'] = np.where(out['WoodDeckSF'] == 0, 0, 1)

    return out

def Summarise_Variables(data:pd.DataFrame) -> pd.DataFrame:

    out = data.copy()

    out['BsmtBathrooms'] = out['BsmtFullBath'] + 0.5*out['BsmtHalfBath']
    out = out.drop(columns=['BsmtFullBath', 'BsmtHalfBath'])

    out['Bathrooms'] = out['FullBath'] + 0.5*out['HalfBath']
    out = out.drop(columns=['FullBath', 'HalfBath'])

    #Age relative to youngest Object
    out['Age'] = out['YearBuilt'].max() - out['YearBuilt']
    out['RemodeledAge'] = np.where(out['Remodeled'] == 1, out['YearRemodAdd'].max() - out['YearRemodAdd'], -1)
    out['GarageAge'] = out['GarageYrBlt'].max() - out['GarageYrBlt'].fillna(out['GarageYrBlt'].max() + 1) #No Garage = -1
    out = out.drop(columns=['YearBuilt', 'YearRemodAdd', 'GarageYrBlt'])

    def SoldDate(month, year):
        month = str(month)
        if len(month) == 1: month = f'0{month}'
        year = str(year)
        year_month = '-'.join([month, year])
        #integrate datetime formatting?
        return year_month
        

    out['Sold'] = out.apply(lambda x: SoldDate(x['MoSold'], x['YrSold']), axis=1)
    out = out.drop(columns=['MoSold', 'YrSold'])

    return out

def Standardize_NonApplicable(data:pd.DataFrame) -> pd.DataFrame:

    out = data.copy()

    # Fill NaN with -1
    out['LotFrontage'] = out['LotFrontage'].fillna(-1)
    # -1 for all variables, where 0 indicates non-applicability
    for col in ['BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'BsmtBathrooms']:
        out[col] = np.where(out['Has_Basement'] == 0, -1, out[col])
    for col in ['GarageCars', 'GarageArea']:
        out[col] = np.where(out['Has_Garage'] == 0, -1, out[col])
    for col in ['2ndFlrSF', 'Fireplaces', 'MiscVal', 'WoodDeckSF', 'OpenPorchSF']: 
        out[col] = out[col].replace({0:-1})

    return out



data = Noncategorical_Dummies(data)
data = Convert_Ordinal(data)
data = Summarise_Variables(data)
data = Standardize_NonApplicable(data)

#Utilities different for only 1 property
#Above grade living area is sum of 1st and 2nd Floor IN MOST CASES!!!
data = data.drop(columns=['Utilities', 'GrLivArea', 'Id'])
#data = data.fillna(-1)

C:\Users\karli\AppData\Local\Temp\ipykernel_45464\1012490662.py:48: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  out[col] = out[col].replace(mapping)
C:\Users\karli\AppData\Local\Temp\ipykernel_45464\1012490662.py:48: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  out[col] = out[col].replace(mapping)
C:\Users\karli\AppData\Local\Temp\ipykernel_45464\1012490662.py:48: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_obje

In [3]:
import pandas as pd

with open('../data/DatasetCleaned.csv', 'w') as f:
    data.to_csv(f)